In [ ]:
# @title
# https://medium.com/georgian-impact-blog/the-practical-guide-to-llms-flan-t5-6d26cc5f14c0

#https://github.com/pedro-r-marques/tutorial-t5-fine-tune

In [ ]:
# @title
USE_SAME_FEW_SHOTS=True

# INSTALLAZIONE E IMPORT LIBRERIE

In [ ]:
# @title
import pickle

In [ ]:
# @title
!pip install -q pillow==10.0.1 --progress-bar off
!pip install -q langchain[llms] --progress-bar off

  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
# @title
#!pip install --upgrade pip
!pip install -q transformers --progress-bar off # Download e caricamento modelli
!pip install -q bitsandbytes --progress-bar off # Quantizzazione
!pip install -q accelerate --progress-bar off
!pip install -q sentencepiece --progress-bar off

In [ ]:
experiment_name = "2024.01.04-02_T5_large_zero-shot_NEW"

In [ ]:
# @title
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"
input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx"

output_data_path = base_path + "/../Flan-T5/data/output/" + experiment_name
log_filename = base_path + "/../Flan-T5/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [ ]:
# @title
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# @title
# T5 comes in different sizes:

#     t5-small
#     t5-base
#     t5-large (800M di parametri)
#     t5-3b (flan-t5-xl)
#     t5-11b.(flan-t5-xxl)

In [ ]:
# @title
# model_id = "meta-llama/Llama-2-7b-chat-hf"
# model_id = "meta-llama/Llama-2-13b-chat-hf"
# model_id = "google/flan-t5-small" # 60M
# model_id = "google/flan-t5-base" # 220M
model_id = "google/flan-t5-large" # 800M
# model_id = "google/flan-t5-xl" # 3B
# model_id = "google/flan-t5-xxl" # 11B

In [ ]:
# @title
# nbit = 4
# # llamasize = '7B'
# llamasize = '13B'
# temp = 0.1
# outdir = 'data/output/2023.11.13-1_T5_run_completo-large___TEST/'
# outdir = 'data/output/2023.11.22-6_T5_run_completo-base_zero-shot-REBOOT/'
# outfilename = outdir+f'sdg_classification_T5_1020-texts_temp-{temp}_nbit-{nbit}_llamasize-{llamasize}.xlsx'
outfilename = output_data_path+f'/sdg_classification_T5_1020-texts.xlsx'

outfilename

'/content/gdrive/MyDrive/Colab/SDG/data/../Flan-T5/data/output/2024.01.04-02_T5_large_zero-shot_NEW/sdg_classification_T5_1020-texts.xlsx'

In [ ]:
# @title
import pandas as pd
# data = pd.read_excel("data/input/data_sel_4_few_shot_examples.xlsx")
data = pd.read_excel(input_data_path)
# data = data.drop(columns=['Unnamed: 0', 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
# data.iloc[61] #text

In [ ]:
# @title
# CUDA

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id, device_map="auto") #, torch_dtype=torch.float16, load_in_4bit=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# @title
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

from langchain.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

In [ ]:
# @title
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig # Questi moduli sono dedicati al modello LLama2
from transformers import AutoModelForCausalLM, AutoTokenizer # Questi moduli permettono di scaricare qualsiasi modello e tokenizer
import pandas as pd
import torch # PyTorch
import langchain

In [ ]:
# @title
###torch.cuda.empty_cache()

## TEST

In [ ]:
# @title
if USE_SAME_FEW_SHOTS==True:
    data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])
    data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
        (data_sel_4_few_shot_examples.text_id==5614) |
        (data_sel_4_few_shot_examples.text_id==2089) |
        (data_sel_4_few_shot_examples.text_id==5665) #|
#         (data_sel_4_few_shot_examples.text_id==370)  |
#         (data_sel_4_few_shot_examples.text_id==1422) |
#         (data_sel_4_few_shot_examples.text_id==4329) |
#         (data_sel_4_few_shot_examples.text_id==4929)
    ]
else:
    data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0']).sample(5)

data_sel_4_few_shot_examples

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
2319,2319,5665,10.18356/3fe10a08-en,"As a result, both the trade and current accoun...",8,0.0,5.0,1.0000,53
3569,3569,2089,10.18356/ee52a573-en,"This refers to the manufacture, use and dispos...",12,3.0,29.0,0.8125,47
4608,4608,5614,10.2139/SSRN.1406042,The modern definitions of the inchoate offence...,16,0.0,6.0,1.0000,136


In [ ]:
# @title
few_shots = ""

for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
    input_text = "Input text: \""+row['text']+"\"\n"
    output_text = "Class name: SDG "+str(row['sdg'])+'\n\n'

    few_shots += input_text
    few_shots += output_text

print(few_shots)

Input text: "As a result, both the trade and current account deficits increased in 2012. Large current account deficits, despite the slowdown in economic growth, are symptomatic of demand-supply imbalances and a pointer to the urgent need to resolve supply bottlenecks. However, capital flows have been adequate to cover the current account deficit thus far."
Class name: SDG 8

Input text: "This refers to the manufacture, use and disposal of lifestyle goods such as electronics and clothing. Key sectors of high impact include food, housing and transport. In these sectors, wafer, energy and waste can be considered as crosscutting sectors affecting and affected by almost every lifestyle domain."
Class name: SDG 12

Input text: "The modern definitions of the inchoate offences are well known in the criminal law since the Middle Ages both in European-Continent legal systems and in the English Common Law. They were mostly developed by the case-laws of the Star Chamber Court, which was abolished

In [ ]:
# @title
# FEW_SHOT_3 = """Text: This refers to the manufacture, use and disposal of lifestyle goods such as electronics and clothing. Key sectors of high impact include food, housing and transport. In these sectors, wafer, energy and waste can be considered as crosscutting sectors affecting and affected by almost every lifestyle domain.
# Question: What is this text about?
# Answer: N (Ensure sustainable consumption and production patterns)


# Text: The modern definitions of the inchoate offences are well known in the criminal law since the Middle Ages both in European-Continent legal systems and in the English Common Law. They were mostly developed by the case-laws of the Star Chamber Court, which was abolished in 1640. The inchoate offences include three basic offences: Attempt, Conspiracy and Solicitation, but different legal systems defined some other offences as inchoate, such as the Aider and Abettor in Britain after the full validation of the Serious Crimes Act, 2007, c.27, s.44. In this paper it is argued, that there is a very intensive interaction between the fight against terrorism and inchoate offences. The fight against terrorism effects the definition of inchoate offences, and inchoate offences are used as a major instrument of criminal law in the legal fight against terrorism.
# Question: What is this text about?
# Answer: R (Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels)


# Text: As a result, both the trade and current account deficits increased in 2012. Large current account deficits, despite the slowdown in economic growth, are symptomatic of demand-supply imbalances and a pointer to the urgent need to resolve supply bottlenecks. However, capital flows have been adequate to cover the current account deficit thus far.
# Question: What is this text about?
# Answer: H (Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all)

# """

In [ ]:
# @title
# FEW_SHOT_3 = """Text: This refers to the manufacture, use and disposal of lifestyle goods such as electronics and clothing. Key sectors of high impact include food, housing and transport. In these sectors, wafer, energy and waste can be considered as crosscutting sectors affecting and affected by almost every lifestyle domain.
# Question: What is this text about?
# Answer: SDG 12 (Ensure sustainable consumption and production patterns)


# Text: The modern definitions of the inchoate offences are well known in the criminal law since the Middle Ages both in European-Continent legal systems and in the English Common Law. They were mostly developed by the case-laws of the Star Chamber Court, which was abolished in 1640. The inchoate offences include three basic offences: Attempt, Conspiracy and Solicitation, but different legal systems defined some other offences as inchoate, such as the Aider and Abettor in Britain after the full validation of the Serious Crimes Act, 2007, c.27, s.44. In this paper it is argued, that there is a very intensive interaction between the fight against terrorism and inchoate offences. The fight against terrorism effects the definition of inchoate offences, and inchoate offences are used as a major instrument of criminal law in the legal fight against terrorism.
# Question: What is this text about?
# Answer: SDG 16 (Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels)


# Text: As a result, both the trade and current account deficits increased in 2012. Large current account deficits, despite the slowdown in economic growth, are symptomatic of demand-supply imbalances and a pointer to the urgent need to resolve supply bottlenecks. However, capital flows have been adequate to cover the current account deficit thus far.
# Question: What is this text about?
# Answer: SDG 8 (Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all)


# """

In [ ]:
# @title
FEW_SHOT_3 = """Text: This refers to the manufacture, use and disposal of lifestyle goods such as electronics and clothing. Key sectors of high impact include food, housing and transport. In these sectors, wafer, energy and waste can be considered as crosscutting sectors affecting and affected by almost every lifestyle domain.
Class: SDG 12 (Ensure sustainable consumption and production patterns)

Text: The modern definitions of the inchoate offences are well known in the criminal law since the Middle Ages both in European-Continent legal systems and in the English Common Law. They were mostly developed by the case-laws of the Star Chamber Court, which was abolished in 1640. The inchoate offences include three basic offences: Attempt, Conspiracy and Solicitation, but different legal systems defined some other offences as inchoate, such as the Aider and Abettor in Britain after the full validation of the Serious Crimes Act, 2007, c.27, s.44. In this paper it is argued, that there is a very intensive interaction between the fight against terrorism and inchoate offences. The fight against terrorism effects the definition of inchoate offences, and inchoate offences are used as a major instrument of criminal law in the legal fight against terrorism.
Class: SDG 16 (Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels)

Text: As a result, both the trade and current account deficits increased in 2012. Large current account deficits, despite the slowdown in economic growth, are symptomatic of demand-supply imbalances and a pointer to the urgent need to resolve supply bottlenecks. However, capital flows have been adequate to cover the current account deficit thus far.
Class SDG 8 (Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all)


"""

In [ ]:
# @title
data_sel = data
# data_sel = data.sample(10)

In [ ]:
# @title
# options_ = """A: End poverty in all its forms everywhere.
# B: End hunger, achieve food security and improved nutrition and promote sustainable agriculture.
# C: Ensure healthy lives and promote well-being for all at all ages.
# D: Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all.
# E: Achieve gender equality and empower all women and girls.
# F: Ensure availability and sustainable management of water and sanitation for all.
# G: Ensure access to affordable, reliable, sustainable and modern energy for all.
# H: Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all.
# I: Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation.
# L: Reduce inequality within and among countries.
# M: Make cities and human settlements inclusive, safe, resilient and sustainable.
# N: Ensure sustainable consumption and production patterns.
# O: Take urgent action to combat climate change and its impacts.
# P: Conserve and sustainably use the oceans, seas and marine resources for sustainable development.
# Q: Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss.
# R: Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels.
# S: Other

# """

# options_

In [ ]:
# @title
options_ = "SDG_1 (End poverty in all its forms everywhere), \
SDG_2 (End hunger, achieve food security and improved nutrition and promote sustainable agriculture), \
SDG_3 (Ensure healthy lives and promote well-being for all at all ages), \
SDG_4 (Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all), \
SDG_5 (Achieve gender equality and empower all women and girls), \
SDG_6 (Ensure availability and sustainable management of water and sanitation for all), \
SDG_7 (Ensure access to affordable, reliable, sustainable and modern energy for all), \
SDG_8 (Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all), \
SDG_9 (Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation), \
SDG_10 (Reduce inequality within and among countries), \
SDG_11 (Make cities and human settlements inclusive, safe, resilient and sustainable), \
SDG_12 (Ensure sustainable consumption and production patterns), \
SDG_13 (Take urgent action to combat climate change and its impacts), \
SDG_14 (Conserve and sustainably use the oceans, seas and marine resources for sustainable development), \
SDG_15 (Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss), \
SDG_16 (Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels), \
SDG_0 (Other)."

In [ ]:
# @title
# options_ = "A (End poverty in all its forms everywhere), \
# B (End hunger, achieve food security and improved nutrition and promote sustainable agriculture), \
# C (Ensure healthy lives and promote well-being for all at all ages), \
# D (Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all), \
# E (Achieve gender equality and empower all women and girls), \
# F (Ensure availability and sustainable management of water and sanitation for all), \
# G (Ensure access to affordable, reliable, sustainable and modern energy for all), \
# H (Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all), \
# I (Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation), \
# L (Reduce inequality within and among countries), \
# M (Make cities and human settlements inclusive, safe, resilient and sustainable), \
# N (Ensure sustainable consumption and production patterns), \
# O (Take urgent action to combat climate change and its impacts), \
# P (Conserve and sustainably use the oceans, seas and marine resources for sustainable development), \
# Q (Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss), \
# R (Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels), \
# S (Other).

# "

In [ ]:
# @title
# # vedi https://www.ibm.com/docs/en/watsonx-as-a-service?topic=lab-sample-prompts#sample1

# template_part1 = """Classify a text into one of 17 classes: SDG 1, SDG 2, SDG 3, SDG 4, SDG 5, SDG 6, SDG 7, SDG 8, SDG 9, SDG 10, SDG 11, SDG 12, SDG 13, SDG 14, SDG 15, SDG 16, SDG 0.

# Class name: SDG 1
# Description: A text about ending poverty in all its forms everywhere

# Class name: SDG 2
# Description: A text about ending hunger, achieving food security and improved nutrition, and promoting sustainable agriculture.

# Class name: SDG 3
# Description: A text about ensuring healthy lives and promoting well-being for all at all ages.

# Class name: SDG 4
# Description: A text about ensuring inclusive and equitable quality education and promoting lifelong learning opportunities for all.

# Class name: SDG 5
# Description: A text about achieving gender equality and empowering all women and girls.

# Class name: SDG 6
# Description: A text about ensuring the availability and sustainable management of water and sanitation for all.

# Class name: SDG 7
# Description: A text about ensuring access to affordable, reliable, sustainable, and modern energy for all.

# Class name: SDG 8
# Description: A text about promoting sustained, inclusive, and sustainable economic growth, full and productive employment, and decent work for all.

# Class name: SDG 9
# Description: A text about building resilient infrastructure, promoting inclusive and sustainable industrialization, and fostering innovation.

# Class name: SDG 10
# Description: A text about reducing inequality within and among countries.

# Class name: SDG 11
# Description: A text about making cities and human settlements inclusive, safe, resilient, and sustainable.

# Class name: SDG 12
# Description: A text about ensuring sustainable consumption and production patterns.

# Class name: SDG 13
# Description: A text about taking urgent action to combat climate change and its impacts.

# Class name: SDG 14
# Description: A text about conserving and sustainably using the oceans, seas, and marine resources for sustainable development.

# Class name: SDG 15
# Description: A text about protecting, restoring, and promoting sustainable use of terrestrial ecosystems, sustainably managing forests, combating desertification, halting and reversing land degradation, and halting biodiversity loss.

# Class name: SDG 16
# Description: A text about promoting peaceful and inclusive societies for sustainable development, providing access to justice for all, and building effective, accountable, and inclusive institutions at all levels.

# Class name: SDG 0
# Description: A text about any other topic different respect the previous ones.
# \n\n
# """+few_shots+"""

# Input text: """#\"{text}\"
# # Class name: """

In [ ]:
# @title
# print(template)

In [ ]:
# @title
# https://github.com/google-research/FLAN/blob/main/flan/v2/templates.py (riga 578)
# "{title}\n\n{text}\n\nMulti-choice problem: What is this text about?\n{options_}"

In [ ]:
# @title
# # https://github.com/google-research/FLAN/blob/main/flan/v2/templates.py (riga 2193)

# # """Patterns for few-shot tasks.

# # The few-shot input are composed by a few examplers followed by final_suffix:
# # {exampler no. 1} + {exampler no. 2} + {exampler no. 3}... + {final_suffix}

# # Each exampler has the following format:
# # {inputs_prefix} + {inputs} + {x_y_delimiter} + {targets_prefix} + {targets} +
# # {example_separator}
# # """

# # # riga 4450
# # FewShotPattern(
# #     inputs="{text}\n\nWhat is a good title for this?",
# #     targets="{title}",
# #     inputs_prefix="Problem: ",
# #     targets_prefix="Title: ",
# #     x_y_delimiter="\n\n",
# #     example_separator="\n\n\n",
# #     in_template_mix=False),

# ex_test_1="This refers to the manufacture, use and disposal of lifestyle goods such as electronics and clothing. Key sectors of high impact include food, housing and transport. In these sectors, wafer, energy and waste can be considered as crosscutting sectors affecting and affected by almost every lifestyle domain."
# ex_test_1_sdg="SDG 12"

# ex_test_2="The modern definitions of the inchoate offences are well known in the criminal law since the Middle Ages both in European-Continent legal systems and in the English Common Law. They were mostly developed by the case-laws of the Star Chamber Court, which was abolished in 1640. The inchoate offences include three basic offences: Attempt, Conspiracy and Solicitation, but different legal systems defined some other offences as inchoate, such as the Aider and Abettor in Britain after the full validation of the Serious Crimes Act, 2007, c.27, s.44. In this paper it is argued, that there is a very intensive interaction between the fight against terrorism and inchoate offences. The fight against terrorism effects the definition of inchoate offences, and inchoate offences are used as a major instrument of criminal law in the legal fight against terrorism."
# ex_test_2_sdg="SDG 16"

# ex_test_3="As a result, both the trade and current account deficits increased in 2012. Large current account deficits, despite the slowdown in economic growth, are symptomatic of demand-supply imbalances and a pointer to the urgent need to resolve supply bottlenecks. However, capital flows have been adequate to cover the current account deficit thus far."
# ex_test_3_sdg="SDG 8"


# inputs_prefix="Input: "
# inputs=f"{ex_test_1}\n\nWhat is a good SDG (Sustainable Development Goal) for this?"
# x_y_delimiter="\n\n"
# targets_prefix="Output: "
# targets=ex_test_1_sdg
# example_separator="\n\n\n"

# example1 = f"{inputs_prefix}{inputs}{x_y_delimiter}{targets_prefix}{targets}{example_separator}"

# inputs=f"{ex_test_2}\n\nWhat is a good SDG (Sustainable Development Goal) for this?"
# targets=ex_test_2_sdg
# example2 = f"{inputs_prefix}{inputs}{x_y_delimiter}{targets_prefix}{targets}{example_separator}"

# inputs=f"{ex_test_3}\n\nWhat is a good SDG (Sustainable Development Goal) for this?"
# targets=ex_test_3_sdg
# example3 = f"{inputs_prefix}{inputs}{x_y_delimiter}{targets_prefix}{targets}{example_separator}"

# examples = example1+example2+example3

# print(examples)

In [ ]:
# @title
# print(examples+"Input: "+text.replace('\n','')+"\n\nWhat is a good SDG (Sustainable Development Goal) for this?\n\nOutput: ")

In [ ]:
text_ids_with_errors = [6145,
 4097,
 6150,
 2055,
 4104,
 6153,
 2056,
 18,
 2068,
 20,
 4130,
 6194,
 57,
 2107,
 60,
 4158,
 63,
 64,
 69,
 78,
 2127,
 6224,
 4176,
 6228,
 4183,
 2143,
 6242,
 6243,
 4198,
 2157,
 4205,
 116,
 2166,
 4214,
 126,
 4226,
 2179,
 4235,
 2190,
 6287,
 4240,
 4254,
 6305,
 6307,
 2212,
 172,
 173,
 2226,
 2228,
 6329,
 2234,
 6331,
 2236,
 6330,
 193,
 2243,
 6340,
 6342,
 202,
 204,
 6351,
 4304,
 211,
 2261,
 4311,
 2267,
 220,
 229,
 2277,
 6378,
 4331,
 2298,
 4349,
 4353,
 262,
 2312,
 6409,
 4363,
 4366,
 4375,
 280,
 2329,
 6425,
 283,
 2335,
 6434,
 4394,
 4396,
 4398,
 4400,
 2354,
 6454,
 322,
 2370,
 334,
 4430,
 4433,
 6481,
 339,
 2391,
 6496,
 359,
 4456,
 6504,
 6506,
 365,
 372,
 2423,
 379,
 4485,
 391,
 4489,
 6541,
 2455,
 6555,
 4509,
 6558,
 2463,
 2469,
 4520,
 2475,
 6575,
 432,
 6582,
 6583,
 4552,
 2512,
 6608,
 2522,
 6626,
 6628,
 2543,
 6656,
 518,
 6665,
 6668,
 2573,
 524,
 535,
 2587,
 2589,
 6698,
 6699,
 4654,
 2608,
 562,
 4660,
 6710,
 567,
 569,
 4673,
 4683,
 588,
 4687,
 2640,
 2639,
 6738,
 598,
 4695,
 6744,
 2652,
 6755,
 611,
 2661,
 2673,
 627,
 6772,
 2676,
 2693,
 2694,
 6796,
 2705,
 4754,
 660,
 2716,
 4769,
 2723,
 678,
 2726,
 4776,
 2735,
 4797,
 2761,
 714,
 2771,
 2775,
 4832,
 4836,
 2791,
 4843,
 4844,
 2798,
 2803,
 759,
 760,
 4861,
 768,
 2818,
 4875,
 2834,
 789,
 4891,
 4896,
 802,
 2852,
 2861,
 2870,
 827,
 835,
 2889,
 842,
 848,
 4948,
 2900,
 854,
 862,
 863,
 2911,
 864,
 4966,
 2920,
 878,
 4974,
 2927,
 2926,
 2930,
 2936,
 2949,
 2952,
 907,
 5015,
 925,
 930,
 5026,
 932,
 934,
 2985,
 940,
 943,
 5043,
 973,
 5077,
 995,
 5101,
 3059,
 5109,
 3070,
 1024,
 3075,
 1028,
 5126,
 1033,
 3086,
 1049,
 5147,
 3105,
 5157,
 5175,
 5181,
 1089,
 3138,
 3148,
 3149,
 1104,
 1114,
 3166,
 3167,
 1122,
 5220,
 1131,
 1134,
 5231,
 5246,
 3201,
 3211,
 3214,
 3217,
 5272,
 1182,
 3236,
 5290,
 1199,
 1200,
 1205,
 3256,
 5312,
 5328,
 5343,
 1248,
 1250,
 5361,
 3315,
 5363,
 3317,
 1273,
 1276,
 1285,
 3336,
 1293,
 3341,
 1299,
 1303,
 3351,
 1306,
 5412,
 3377,
 5431,
 5441,
 1346,
 5443,
 1348,
 3397,
 5448,
 5449,
 5450,
 3404,
 3417,
 3418,
 1372,
 1373,
 1376,
 3429,
 1388,
 3438,
 3444,
 3445,
 3448,
 3450,
 5511,
 1427,
 5527,
 5533,
 1444,
 3494,
 1449,
 1453,
 1454,
 1455,
 3501,
 5555,
 5570,
 5575,
 1489,
 3540,
 5589,
 1497,
 5600,
 1506,
 3560,
 1515,
 1516,
 5617,
 3571,
 1524,
 1535,
 5642,
 1547,
 1548,
 1553,
 3604,
 5653,
 5654,
 1559,
 1560,
 3611,
 1565,
 3613,
 3614,
 1568,
 5666,
 1570,
 5668,
 3618,
 5667,
 3626,
 3637,
 5693,
 3653,
 1607,
 1609,
 1613,
 1616,
 3665,
 5720,
 3673,
 5738,
 5742,
 1653,
 3703,
 5761,
 5806,
 5813,
 1725,
 3777,
 1734,
 3782,
 3790,
 3791,
 1742,
 5851,
 5859,
 1767,
 1776,
 1781,
 1786,
 3839,
 1794,
 3844,
 5893,
 3851,
 5922,
 3877,
 5928,
 3883,
 5932,
 1838,
 1840,
 3889,
 3891,
 1851,
 3912,
 1864,
 1866,
 1869,
 5966,
 3918,
 3927,
 3928,
 1881,
 3934,
 5999,
 6007,
 6011,
 3964,
 6039,
 6047,
 4001,
 4014,
 4015,
 1968,
 6073,
 6074,
 6077,
 4039,
 4040,
 1992,
 6094,
 6100,
 4054,
 6104,
 2023,
 2027,
 4078,
 4083,
 4087,
 6136,
 2042]

In [ ]:
len(text_ids_with_errors)

457

In [ ]:
# data_sel = data #.iloc[739:] #.sample(10)
data_sel = data[data.text_id.isin(text_ids_with_errors)] #.iloc[:10]

In [ ]:
%%time

import time

results = []
# cnt = 0
# cnt = 740

for index, row in data_sel.iterrows():

    # AGGIUNGO QUESTO WHILE PER RIPETERE L'INFERENZA FINCHE' IL MODELLO NON
    # RESTITUISCE QUALCOSA DI SENSATO
    # while True:

    file = open(log_filename, 'a+', encoding='utf-8')

    cnt = index

    text = row['text']
    text_id = row['text_id']
    sdg = row['sdg']

    # template = FEW_SHOT_3+f"Text: {text}\n\nWhat is this text about?\n\n{options_}"
#     template = FEW_SHOT_3+f"""Text: {text}
# Question: What is this text about?
# Answer: """

    # template = FEW_SHOT_3+f"""Text: """+text.replace('\n','')+"""\nClass:"""

    template = "Please classify the following text based on the UN's Sustainable Development Goals (SDGs) outlined in the Agenda 2030. Use the following labels SDG_1, SDG_2, ..., SDG_17 for the specific classes and SDG_0 for the text that are not classifiable in this domain. The text is: "+text.replace('\n','').replace('{','(').replace('}',')')

    # template = "Classifica il seguente testo secondo l'obiettivo di sviluppo sostenibile dell'Agenda 2030 dell'ONU più rilevante. Utilizza le etichette SDG_1, SDG_2, ..., SDG_17 per gli obiettivi specifici, e SDG_0 per i testi che non sono classificabili in questo dominio."

    # template = "Please classify the following text based on its alignment with the UN's Sustainable Development Goals (SDGs) outlined in the Agenda 2030. Identify and describe which goals (from SDG_1 to SDG_16) from the Agenda 2030 this text could contribute to achieving; use the label SDG_0 for the 'Other' class. The text is: "+text.replace('\n','').replace('{','(').replace('}',')')


#     template = """TASK: Given the text provided, determine its classification in the context of the Sustainable Development Goals Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015. Identify the most suitable category among SDG_1 (End poverty in all its forms everywhere), \
# SDG_2 (End hunger, achieve food security and improved nutrition and promote sustainable agriculture), \
# SDG_3 (Ensure healthy lives and promote well-being for all at all ages), \
# SDG_4 (Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all), \
# SDG_5 (Achieve gender equality and empower all women and girls), \
# SDG_6 (Ensure availability and sustainable management of water and sanitation for all), \
# SDG_7 (Ensure access to affordable, reliable, sustainable and modern energy for all), \
# SDG_8 (Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all), \
# SDG_9 (Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation), \
# SDG_10 (Reduce inequality within and among countries), \
# SDG_11 (Make cities and human settlements inclusive, safe, resilient and sustainable), \
# SDG_12 (Ensure sustainable consumption and production patterns), \
# SDG_13 (Take urgent action to combat climate change and its impacts), \
# SDG_14 (Conserve and sustainably use the oceans, seas and marine resources for sustainable development), \
# SDG_15 (Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss), \
# SDG_16 (Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels), \
# SDG_0 (Other).

# TEXT: """+text.replace('\n','').replace('{','(').replace('}',')')+"""
# CLASSIFICATION:"""

#     template = text.replace('\n','').replace('{','(').replace('}',')')+"""\n\nWhat is this text about?\n\nA: End poverty in all its forms everywhere.
# B: End hunger, achieve food security and improved nutrition and promote sustainable agriculture.
# C: Ensure healthy lives and promote well-being for all at all ages.
# D: Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all.
# E: Achieve gender equality and empower all women and girls.
# F: Ensure availability and sustainable management of water and sanitation for all.
# G: Ensure access to affordable, reliable, sustainable and modern energy for all.
# H: Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all.
# I: Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation.
# L: Reduce inequality within and among countries.
# M: Make cities and human settlements inclusive, safe, resilient and sustainable.
# N: Ensure sustainable consumption and production patterns.
# O: Take urgent action to combat climate change and its impacts.
# P: Conserve and sustainably use the oceans, seas and marine resources for sustainable development.
# Q: Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss.
# R: Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels.
# S: Other"""


    #print("#########################################################################################")
    #print(template)

    # template = template_part1 + f"\"{text}\"\n Class name: "

    # template = examples+"Input: "+text.replace('\n','')+"\n\nWhat is a good SDG (Sustainable Development Goal) for this?\n\nOutput: "

    #print(template)
    try:
      input_ids = tokenizer(template, return_tensors="pt").input_ids.to("cuda")
      outputs = model.generate(input_ids, temperature=0.9, repetition_penalty=1.2)
      results.append([
          text_id,
          sdg,
          tokenizer.decode(outputs[0])
      ])

      # result_extended = (cnt, text_id, sdg, tokenizer.decode(outputs[0]))

      # print(result_extended)

      # if "SDG" not in tokenizer.decode(outputs[0]):
      #   print(time.ctime(time.time()), (cnt, text_id, sdg, "<pad> SDG_99</s>"), "WARNING: OUTPUT SET TO DEFAULT VALUE")
      #   file.write(str(time.ctime(time.time())) + " " + str(cnt)+ " " +str(text_id)+ " " +str(sdg)+ " " + "<pad> SDG_99</s> -  WARNING: OUTPUT SET TO DEFAULT VALUE" + "\n")
      #   result_extended = (cnt, text_id, sdg, "<pad> SDG_99</s>")
      #   with open(output_data_path+f'/sdg_classification_T5_{cnt}.pkl', 'wb') as f:
      #       pickle.dump(result_extended, f)
      # else:
      print(time.ctime(time.time()), (cnt, text_id, sdg, tokenizer.decode(outputs[0])))
      file.write(str(time.ctime(time.time())) + " " + str(cnt)+ " " +str(text_id)+ " " +str(sdg)+ " " +str(tokenizer.decode(outputs[0])) + "\n")
      result_extended = (cnt, text_id, sdg, tokenizer.decode(outputs[0]))
      with open(output_data_path+f'/sdg_classification_T5_{cnt}.pkl', 'wb') as f:
          pickle.dump(result_extended, f)
    except Exception as e:
      print(time.ctime(time.time()), (cnt, text_id, sdg, "ERROR: "+str(e)))
      file.write(str(time.ctime(time.time())) + " " + str(cnt) + " " + str(text_id) + " " + str(sdg) + " " + str("ERROR: "+str(e)) + "\n")


    file.close()

    #cnt+=1

    # if cnt%100==0:
    #    print(cnt)

    # if cnt==10:
    #     break

    # QUESTA E' LA CONDIZIONE DI USCITA DAL WHILE
    # if "SDG" in tokenizer.decode(outputs[0]):
    #   break

  #print(template)

Thu Jan  4 14:30:57 2024 (0, 6668, 0, '<pad> SDG_0</s>')
Thu Jan  4 14:30:57 2024 (3, 1786, 0, '<pad> SDG_0</s>')
Thu Jan  4 14:30:58 2024 (4, 2143, 0, '<pad> SDG_0</s>')
Thu Jan  4 14:30:58 2024 (5, 3377, 0, '<pad> SDG_0</s>')
Thu Jan  4 14:30:58 2024 (6, 5589, 0, '<pad> SDG_0</s>')
Thu Jan  4 14:30:59 2024 (7, 5431, 0, '<pad> SDG_0</s>')
Thu Jan  4 14:30:59 2024 (8, 768, 0, '<pad> SDG_0</s>')
Thu Jan  4 14:30:59 2024 (9, 2735, 0, '<pad> SDG_0</s>')
Thu Jan  4 14:31:00 2024 (10, 1306, 0, '<pad> SDG_0</s>')
Thu Jan  4 14:31:00 2024 (11, 3844, 0, '<pad> SDG_0</s>')
Thu Jan  4 14:31:01 2024 (12, 1524, 0, '<pad> SDG_0</s>')
Thu Jan  4 14:31:01 2024 (13, 4660, 0, '<pad> SDG_0</s>')
Thu Jan  4 14:31:02 2024 (14, 5077, 0, '<pad> SDG_0</s>')
Thu Jan  4 14:31:02 2024 (15, 995, 0, '<pad> SDG_0</s>')
Thu Jan  4 14:31:02 2024 (16, 2166, 0, '<pad> 0</s>')
Thu Jan  4 14:31:03 2024 (17, 2512, 0, '<pad> SDG_0</s>')
Thu Jan  4 14:31:03 2024 (19, 6073, 0, '<pad> SDG_0</s>')
Thu Jan  4 14:31:03 2024 (20

In [ ]:
# results

In [ ]:
# with open(output_data_path+f'/sdg_classification_T5_13B_{cnt}.pkl', 'wb') as f:
#     pickle.dump(result_extended, f)

In [ ]:
data_sel = data

In [ ]:
import glob

files = glob.glob(output_data_path + "/*.pkl")

results = []
for file in files:
    res = pd.read_pickle(file)
    # if res[3] == "<pad> SDG_99</s>":
    #   res = (res[0], res[1], res[2], "<pad> SDG_0</s>")
    results.append(res)

len(results)

1020

In [ ]:
results_processed = pd.DataFrame(results, columns=['cnt', 'id', 'true_label', 'predicted_label'])
results_processed

,cnt,id,true_label,predicted_label
0,328,3273,5,<pad> SDG_5</s>
1,330,486,5,<pad> SDG_5</s>
2,331,5159,5,<pad> SDG_5</s>
3,333,2109,5,<pad> SDG_0</s>
4,334,378,5,<pad> SDG_0</s>
...,...,...,...,...
1015,320,3902,5,<pad> SDG_5</s>
1016,322,5048,5,<pad> SDG_5</s>
1017,325,2883,5,<pad> SDG_5</s>
1018,326,4899,5,<pad> SDG_5</s>


In [ ]:
# results_processed['label'] = results_processed['predicted_label'].str.extract(r'SDG (\d+)')
results_processed['label'] = results_processed['predicted_label'].str.replace('<pad> ', '')
results_processed['label'] = results_processed['label'].str.replace('</s>', '')

In [ ]:
results_processed.groupby('label').count()

,cnt,id,true_label,predicted_label
label,,,,
0,2,2,2,2
SDG_0,785,785,785,785
SDG_10,23,23,23,23
SDG_11,4,4,4,4
SDG_12,2,2,2,2
SDG_13,22,22,22,22
SDG_14,15,15,15,15
SDG_15,33,33,33,33
SDG_17,3,3,3,3


In [ ]:
# label_list = ['dummy', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S']

# results_processed = results_processed[results_processed.label.str.len() == 1]
# results_processed['label'] = results_processed['label'].apply(lambda x : label_list.index(x))
# results_processed

In [ ]:
results_processed['label'] = results_processed['label'].str.replace("_", " ")
results_processed['label'] = results_processed['label'].str.extract(r'SDG (\d+)')

In [ ]:
final = data_sel.merge(results_processed, left_on='text_id', right_on='id')
final.head(3)

,Unnamed: 0.1,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,label
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,<pad> SDG_0</s>,0
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,<pad> SDG_0</s>,0
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,<pad> SDG_0</s>,0


In [ ]:
# text_ids_with_errors = final[final.label.isna()].text_id.to_list()
text_ids_with_errors = final[final.label=='99'].text_id.to_list()
list(set(text_ids_with_errors))

[]

In [ ]:
#df.loc[df['label'] == 99, 'label'] = 0

In [ ]:
final.to_excel(outfilename)

In [ ]:
final = final[~final.label.isna()]

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

In [ ]:
outdir = output_data_path

In [ ]:
classification_report_filename = outdir+f'sdg_classification_T5_1020-texts_classification_report.xlsx'
classification_report_filename

'/content/gdrive/MyDrive/Colab/SDG/data/../Flan-T5/data/output/2024.01.04-02_T5_large_zero-shot_NEWsdg_classification_T5_1020-texts_classification_report.xlsx'

In [ ]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_excel(classification_report_filename)

In [ ]:
test = pd.read_excel(classification_report_filename)
test

,Unnamed: 0,precision,recall,f1-score,support
0,0,0.075949,1.000000,0.141176,60.000000
1,1,0.000000,0.000000,0.000000,60.000000
2,2,1.000000,0.100000,0.181818,60.000000
3,3,1.000000,0.100000,0.181818,60.000000
4,4,0.909091,0.166667,0.281690,60.000000
5,5,0.964286,0.450000,0.613636,60.000000
6,6,0.944444,0.283333,0.435897,60.000000
7,7,0.870968,0.450000,0.593407,60.000000
8,8,0.454545,0.083333,0.140845,60.000000
9,9,0.700000,0.233333,0.350000,60.000000
